In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
#%matplotlib inline
from datetime import date, datetime
import os
import seaborn as sns
 
from sklearn.linear_model import LinearRegression

# Set figure size to (14,6)
plt.rcParams['figure.figsize'] = (14,6)
# Set some easthetic
sns.set()


In [ ]:
wd = 'Hamburg/'
#df = pd.read_csv(wd + 'hamb_temp.txt', index_col=0, header =0)
df = pd.read_csv(wd + 'hamb_temp.txt', index_col = 0, 
                      parse_dates = True,sep=",")
df['Date_dt'] = df['DATE']
df.set_index('DATE', inplace=True)
#df['Date_dt'] = str(df['DATE'])

In [ ]:
df.head()

In [ ]:
def plot_flights(df, title='Monthly Passenger Numbers in 1000 over Time', ylim=True):
    '''
    Custom plotting function for plotting the flights dataset
    
    Parameters
    ----------
    df : pd.DataFrame
        The data to plot.
    title : str
        The title of the plot
    ylim : bool
        Whether to fix the minimum value of y; defalut is True
    
    Returns
    -------
    Plots the data
    '''
    df.plot()
    plt.title(title)
    plt.ylabel('# Temperature C')
    if ylim:
        plt.ylim(ymin=0)
    plt.show()

In [ ]:
plot_flights(df=df)

In [ ]:
df['Date_dt'] = pd.to_datetime(df['Date_dt'], format='%Y%m%d')


In [ ]:
df.dtypes


In [ ]:
df.drop(df.columns[[1]], axis=1, inplace=True)

In [ ]:
#df['TG'] = df['TG'] / 10
df.rename(columns = {'TG' : 'Avg_temp'}, inplace = True)
df['Avg_temp'] = df['Avg_temp'] / 10


In [ ]:
ts = df['Avg_temp']
ts.head(10)

In [ ]:
plt.plot(ts)

In [ ]:
ts.hist(bins=200)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=30)
    rolstd = pd.rolling_std(timeseries, window=30)
#Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
ts.plot(kind='kde')

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.index  =df['Date_dt']

In [ ]:
#create new date columns based on the 'datetime' string column
df['Date_dt'] = pd.to_datetime(df['Date_dt'])
 
#year
from calendar import month_name

df['year'] = df['Date_dt'].dt.year
#Month
df['month'] = df['Date_dt'].dt.month
df['day_month'] = df['Date_dt'].dt.month_name()

#day
df['day'] = df['Date_dt'].dt.day
df['day_name'] = df['Date_dt'].dt.day_name()


In [ ]:
##Add a timestep feature
df['timestep'] = range(len(df))

In [ ]:
###make dummy variables for year
seasonal_dummies = pd.get_dummies(df.index.year,
                                  prefix='year',
                                  drop_first=False).set_index(df.index)

df = df.join(seasonal_dummies)

In [ ]:
###make dummy variables for month
seasonal_dummies2 = pd.get_dummies(df.index.month,
                                  prefix='month',
                                  drop_first=False).set_index(df.index)

df = df.join(seasonal_dummies2)

In [ ]:
####create a small test dataset by removing all 2022 data points

#test = df.loc[df.index >= '2021-10-01 00:00:00']
#training = df.loc[df.index < '2021-10-01 00:00:00']

test = df.loc[df.index >= '2022-06-01 00:00:00']
training = df.loc[df.index < '2022-06-01 00:00:00']



In [ ]:
##make a baseline model called model m
#######################################################################
X = training[['timestep']]
y = training['Avg_temp']

m = LinearRegression()
m.fit(X,y)

In [ ]:
training['ts_trend'] = m.predict(X) 
training

In [ ]:
plot_flights(df=training[['Avg_temp','ts_trend']])

In [ ]:
##  Model the seasonality
######generate model called model m2

In [ ]:
training.columns

In [ ]:
X2 = training.drop(['Avg_temp', 'month', 'year','day_month' , 'day', 'day_name','ts_trend'],axis=1) 
X2.set_index('Date_dt', inplace=True)
y2= training['Avg_temp']
m2 = LinearRegression()
m2.fit(X2,y2)

In [ ]:
training['trend_with_seasonal'] = m2.predict(X2)
training.head(10)

In [ ]:
plot_flights(training[['Avg_temp','trend_with_seasonal']])

In [ ]:
training

In [ ]:
#Build alternate m2 model by removing Year dummy value

In [ ]:

X2_my = training[['timestep','month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 
'month_10', 'month_11', 'month_12']]
#X2_my.index

In [ ]:
###create revises seasonal model but remove the year dummy data

X2_my = training[['timestep','month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 
'month_10', 'month_11', 'month_12']]
#X2_my.set_index('Date_dt', inplace=True)
y2_my= training['Avg_temp']
m2_my = LinearRegression()
m2_my.fit(X2_my,y2_my)

In [ ]:
training['trend_with_seasonal'] = m2_my.predict(X2_my)
training.head(10)

In [ ]:
plot_flights(training[['Avg_temp','trend_with_seasonal']])

In [ ]:
## - Extract the remainder
########################################

In [ ]:
training['remainder'] = training['Avg_temp'] - training['trend_with_seasonal']

In [ ]:
plot_flights(training['remainder'],title = "Remainder",ylim=False)

In [ ]:
######Use remainder to assess the numner of lags required for final model

In [ ]:
#from pandas.io.json.normalize import nested_to_record
training['remainder'].to_csv('flights_remainder.csv')

In [ ]:
#load in remainder file
df_rem = pd.read_csv('flights_remainder.csv')

In [ ]:

df_rem['lag1'] = df_rem['remainder'].shift(1)

In [ ]:
df_rem.corr()

In [ ]:
sns.scatterplot(x='lag1', y='remainder', data=df_rem);

In [ ]:
df_rem.dropna(inplace=True)
df_rem.head()

In [ ]:
df_rem['lag2'] = df_rem['remainder'].shift(2)

In [ ]:
df_rem.dropna(inplace=True)
df_rem.head()

In [ ]:
sns.scatterplot(x='lag2', y='remainder', data=df_rem);

In [ ]:
df_rem['lag3'] = df_rem['remainder'].shift(3)
df_rem.dropna(inplace=True)
df_rem.head()

In [ ]:
df_rem.dropna(inplace=True)
df_rem.head()

In [ ]:
sns.scatterplot(x='lag3', y='remainder', data=df_rem);

In [ ]:
####################################################
#################################

In [ ]:

##build model with 3 lags

#Run an Autoregression (Linear Regression) of lag1 on the remainder

In [ ]:
X_l3 = df_rem[['lag1','lag2' ,'lag3' ]]
y_l3 = df_rem['remainder']

In [ ]:
# Create and fit the model calle arm3
arm3 = LinearRegression()
arm3.fit(X_l3 , y_l3)
# Create predictions
df_rem['predictions_ar1'] = arm3.predict(X_l3)
df_rem

In [ ]:
arm3.coef_

In [ ]:
df_rem.columns

In [ ]:
###Assess what number of lags is needed with pacf plot

#from statsmodels.graphics.tsaplots import plot_acf  
from statsmodels.graphics.tsaplots import plot_pacf
#import plot_pacf
plot_pacf(df_rem['remainder'])
plt.xlabel('lags');

In [ ]:
###Build final full model with the 3lags
##################################################################
#########################################################################

In [ ]:
##create the 3 lags in the training dataset that were explored in the previous section

training['lag1'] = training['remainder'].shift(1)
training.dropna(inplace=True)
training['lag2'] = training['remainder'].shift(2)
training.dropna(inplace=True)
training['lag3'] = training['remainder'].shift(3)
training.dropna(inplace=True)
training.head() 

In [ ]:
# Assign X_full and build model
X_full = training.drop(columns=['Avg_temp', 'month', 'year','day_month' , 'day', 'day_name'
, 'trend_with_seasonal', 'remainder', 'ts_trend'],axis=1)
X_full.set_index('Date_dt', inplace=True)
y_full = training['Avg_temp']

In [ ]:
###fit the final model m3
######################################################################################
############################################################################################
m3 = LinearRegression()
m3.fit(X_full,y_full)

In [ ]:
training['predictions_full_model'] = m3.predict(X_full)
training

In [ ]:
#choose model with 3 lags only called model m3
#use the training dataset since we now know that we need 3 lags of the remainder
##recreate 3 lags and run the full model after our previous investigations 

In [ ]:
training.columns

In [ ]:
##Some extraneous Model dataset cleanup that is redundant
####drop the 'predictions_full_model' column for the cross validation phase
'''X_full = X_full.drop(columns=['predictions_full_model'],axis=1)
X_full.tail()
X_full = X_full.drop(columns=['year_2022'],axis=1)
X_full = X_full.loc[X_full.index >= '1937-01-01 00:00:00']
y_full = y_full.loc[y_full.index >= '1937-01-01 00:00:00']
'''

In [ ]:
###run statsmodel ARIMA models#############
#################################

In [ ]:
from statsmodels.api import OLS, add_constant, qqplot
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.graphics.tsaplots import plot_pacf

selected_order = ar_select_order(training['remainder'], maxlag = 5)

In [ ]:
print(f"The lag we need to include are {selected_order.ar_lags}")

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

ar_model = AutoReg(endog=training['remainder'], lags=3).fit()
ar_model.summary()

In [ ]:
ar_model3 = AutoReg(endog=training['remainder'], lags=10).fit()
ar_model3.summary()

In [ ]:
###final model chosen is model with 3 lags: either ar_model3 or m3 

In [ ]:
####Evaluating Forecast###############################################################
##########################################

In [ ]:
###Model 3 comparison 
plot_flights(training[['Avg_temp',  'predictions_full_model' ,'trend_with_seasonal']])

In [ ]:
##model 3 final coefficients
m3.coef_

In [ ]:
#CROSS VALIDATION OF THE FINAL m3 ARIMA MODEL
#######################################################################

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, KFold
from statsmodels.tsa.ar_model import AutoReg



In [ ]:
plt.rcParams['figure.figsize'] = (14,6)

In [ ]:
#Could not get this For loop to work and generate the correct Cross validation
# Split the training data into folds
#ts_split = TimeSeriesSplit(n_splits=10)
''''for i, (train_index, validation_index) in enumerate(ts_split.split(X_full, y_full)):
    print(f"""The training data for the {i+1}th iteration are the observations steps 
    {train_index}""")
    print(f"""The validation data for the {i+1}th iteration are the observations steps
    {validation_index}""")
    print('\n')
    '''

In [ ]:
####drop the 'predictions_full_model' column for the cross validation phase
#X_full = X_full.drop(columns=['predictions_full_model'],axis=1)

In [ ]:
# Set the Cross Validation parameters with randomisation
#time_series_split = ts_split.split(X_full, y_full,) 
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
# Do the cross validation: Remember these are the 'test scores' in the training data.
result = cross_val_score(estimator=m3,
                         X=X_full, y=y_full, 
                         cv=cv)
result

In [ ]:
result.mean()

In [ ]:
# Test the final aRIMA  model m3 with 3 lags###########################
###############################################
##########################################

In [ ]:
test.head()

In [ ]:
# Get last timestep of the training data
last_train_timestep = X_full['timestep'][-1]

In [ ]:
#last_train_timestep = training['timestep'][-1]

In [ ]:
last_train_timestep 

In [ ]:
# Create a timestep for the whole test data
test['timestep'] = list(range(last_train_timestep + 1, 
                            last_train_timestep + len(test) + 1))
test.head()

In [ ]:
#set index to Date_dt 
test.set_index('Date_dt', inplace=True)

In [ ]:
test.columns

In [ ]:
###dummy variables for Year and Month are already in place for the test set as they got created before the original split
####Drop Avg_temp and some other variables from test dataset:

y_test = test['Avg_temp'] ##
X_test = test.drop([ 'month', 'year','day_month' , 'day', 'day_name'],axis=1) 


#Predict trend and seasonality for the test using final arima model **m** (which is trend_seasonal ##model for the training).

*Use model m generated using training set to predict  and add 'ts_trend' o  the test dataset
*Use model m2 generated using training set to predict and add 'trend_with_seasonal' to the test dataset



In [ ]:
#y_test[['Avg_temp']]

In [ ]:
#Use model m generated using training set to predict  and add 'ts_trend' o  the test dataset

X_test = test[['timestep']]
#y_test = y_test['Avg_temp']

test['ts_trend'] = m.predict(X_test) #Xtest should have same number of columns as Dataset X
test.head()


In [ ]:
#Use model m2 generated using training set to predict and add 'trend_with_seasonal' to the test dataset
#m2_my.predict(X2_my) alternate model is m2_my with not year dummies

X_test = test.drop([ 'Avg_temp','month', 'year','day_month' , 'day', 'day_name', 'ts_trend'],axis=1) 
#X_test should look liek X2 dataset
test['trend_with_seasonal'] = m2.predict(X_test) 
test.head()

In [ ]:
test.head(5)

In [ ]:
plot_flights(test[['Avg_temp', 'trend_with_seasonal']], ylim=False)

In [ ]:
X_test_my = test[['timestep','month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 
'month_10', 'month_11', 'month_12']]

In [ ]:

X_test_my = test[['timestep','month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 
'month_10', 'month_11', 'month_12']]
#X_test should look liek X2 dataset
test['trend_with_seasonal'] = m2_my.predict(X_test_my) 
test.head()

In [ ]:
plot_flights(test[['Avg_temp', 'trend_with_seasonal']], ylim=False)

In [ ]:
##Calculate the remainder for the test set.

In [ ]:
test['remainder'] = test['Avg_temp'] - test['trend_with_seasonal']

And finally the lag for the remainder as our additional feature for the AR model.

In [ ]:
test['lag1'] = test['remainder'].shift(1)
training.dropna(inplace=True)
test['lag2'] = test['remainder'].shift(2)
training.dropna(inplace=True)
test['lag3'] = test['remainder'].shift(3)
training.dropna(inplace=True)

In [ ]:
last_n_rows = X_full.iloc[-3:]

In [ ]:
last_n_rows[['timestep','lag1','lag2','lag3']]

In [ ]:
last_n_rows

In [ ]:
#end of the trainin dataset, use these values to fill in the missing lag value for the test set first 3 rows
last_n_rows.iloc[0:,100:103]  
last_n_rows.iloc[2,100]   ##replacement value for the lag1 in the dataset
last_n_rows.iloc[1,101]   ##replacement value for the first missing value of lag2 in the dataset
last_n_rows.iloc[2,101]   ##replacement value for the the first missing value of  lag2 in the dataset
last_n_rows.iloc[0,101]   ##replacement value for the the first missing value of  lag2 in the dataset
last_n_rows.iloc[1,102]   ##replacement value for the the second missing value of  lag2 in the dataset
last_n_rows.iloc[2,102]   ##replacement value for the the first missing value of  lag2 in the dataset

In [ ]:
#fill in the lag null values in the test dataset

In [ ]:
test.iloc[-3:]

In [ ]:
############Replace the test set 3 lags with the last values from the training set
test.loc['2022-06-01', 'lag1']  
test.loc['2022-06-01', 'lag1'] = last_n_rows.iloc[2,100] ##replce lag 1 first value
test.loc['2022-06-01', 'lag2'] = last_n_rows.iloc[0,101] ##replce lag 2 first value
test.loc['2022-06-01', 'lag3'] = last_n_rows.iloc[0,102] ##replce lag 3 first value

test.loc['2022-06-02', 'lag2'] = last_n_rows.iloc[2,102]  ##replce lag 2 second  value

test.loc['2022-06-02', 'lag3'] = last_n_rows.iloc[1,102]  ##replce lag 3 second value
test.loc['2022-06-03', 'lag3'] = last_n_rows.iloc[2,102]   ##replce lag 3 third value


In [ ]:
#X_test = test.drop([ 'Avg_temp','month', 'year','day_month' , 'day', 'day_name', 'ts_trend'],axis=1) 

In [ ]:
# create the test datset for the m3 model
#X_full_test = test.drop(columns=['predictions_full_model'],axis=1)

In [ ]:
X_full_test = test.drop(['remainder', 'trend_with_seasonal', 'Avg_temp','month', 'year','day_month' , 'day', 'day_name', 'ts_trend'],axis=1) 

In [ ]:
# Create the predictions
test['predictions_full_model'] = m3.predict(X_full_test)## X_full_test should look like X_full

In [ ]:
plot_flights(test[['Avg_temp', 'trend_with_seasonal', 'predictions_full_model']], ylim=False)

In [ ]:
# Create the complete dataset and plot it
temp_full = training[['Avg_temp', 'trend_with_seasonal', 'predictions_full_model']].append(test[['Avg_temp', 'trend_with_seasonal', 'predictions_full_model']])

In [ ]:
temp_full.head(-5)

In [ ]:
plot_flights(temp_full)

In [ ]:
X_test.head(1)

In [ ]:
##calculate test score for model

In [ ]:
print(f"""
full model = {m3.score(X_full_test, test['Avg_temp'])} #X_full_test should look like X_full
trend_seasonal = {m2.score(X_test, test['Avg_temp'])} 
""") ##X_test should look like  X2

In [ ]:
# 13) - Predict the future